# Feature Extraction and Modeling

# General Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision.transforms as transforms

from torchvision.models import resnet50, ResNet50_Weights
from torch.utils.data import random_split


from torch.utils.data import DataLoader
import os
import sklearn.model_selection as model_selection
import sklearn.linear_model as linear_model
import subprocess
from tqdm import tqdm

from sklearn.decomposition import PCA, KernelPCA

from sklearn.neural_network import MLPClassifier

from methods import (
    get_labels,
    ImageHeuristicFeatureExtractor,
    standardize_features,
    ImageDataset,
    merge_features_with_labels,
    not_oversampled_images,
    calculate_test_size,
    plot_confusion_matrix,
    plot_low_dim_components,
)

from data_augmentation import split_data_and_oversample

from sklearn.preprocessing import StandardScaler
from sklearn import svm

from sklearn.manifold import TSNE
import shap

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

import torch.nn as nn
from torch.utils.data import Dataset
from PIL import Image


%load_ext autoreload
%autoreload 2

In [ ]:
# load repo and
repo_dir = (
    subprocess.Popen(["git", "rev-parse", "--show-toplevel"], stdout=subprocess.PIPE)
    .communicate()[0]
    .rstrip()
    .decode("utf-8")
)

# Set the folder containing the raw images
original_folder_path = os.path.join(
    repo_dir, "dataverse_files/HAM10000_images_part_1_2_3"
)

# Create Folders
features_folder_path = os.path.join(repo_dir, "features_extracted")
processed_folder_path = os.path.join(repo_dir, "preprocessed_images")
figures_folder_path = os.path.join(repo_dir, "figures")
os.makedirs(processed_folder_path, exist_ok=True)
os.makedirs(features_folder_path, exist_ok=True)
os.makedirs(figures_folder_path, exist_ok=True)

# get pandas dataframe
label = get_labels(repo_dir)

# Load old features (or write new features using this name)
cnn_features_path = os.path.join(features_folder_path, "features.json")

TRAIN_SIZE = 0.8
OVERSAMPLE = False  #  if set to false will not oversample the minority class

if "oversampled" in cnn_features_path and OVERSAMPLE:
    data_folder_path = processed_folder_path
else:
    data_folder_path = original_folder_path

# Oversampling

In [ ]:
if OVERSAMPLE:
    if not os.listdir(data_folder_path):
        try:
            split_data_and_oversample(
                original_folder_path,
                processed_folder_path,
                label,
                TRAIN_SIZE,
                oversample=OVERSAMPLE,
                move_picture_up_levels=2,
            )
        except OSError as e:
            print(e)
            assert False, "delete the Folder 'preprocessed_images' and try again"
    else:
        print(f"Used pre-processed features at {data_folder_path}")
else:
    print("No Oversampling")

# Feature Extraction

## (A) Feature Exctraction using ResNet50 (CNN)

In [ ]:
# Using the RestNet 50 model to extract features using pretrained weights
model = resnet50(weights=ResNet50_Weights.DEFAULT)

In [ ]:
transform = transforms.Compose(
    [
        transforms.Resize(232),
        transforms.CenterCrop(450),  # adapted to use larger region
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)
dataset = ImageDataset(directory=data_folder_path, transform=transform)
data_loader = DataLoader(dataset, batch_size=16, shuffle=False, num_workers=0)

In [ ]:
# Determine the best available device
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"
print(f"Using device: {device}")

In [ ]:
if not os.path.exists(cnn_features_path):
    model = model.to(device)  # Move your model to the appropriate device
    model.eval()  # Set the model to evaluation mode

    features_map2 = {}
    with torch.no_grad():
        for batch_idx, (key, images) in enumerate(tqdm(data_loader)):
            images = images.to(device)  # Move images to the appropriate device

            batch_features = model(images)
            batch_features = batch_features.view(
                batch_features.size(0), -1
            )  # Flatten features

            batch_features = (
                batch_features.cpu().numpy()
            )  # Move features to CPU for numpy conversion

            for i, feature in enumerate(batch_features):
                image_id = (
                    batch_idx * data_loader.batch_size + i
                )  # Compute global image ID/index
                features_map2[key[i]] = feature

    # Saving the raw features
    features_df = pd.DataFrame(features_map2)
    features_df.to_json(cnn_features_path)
else:
    print(f"Previously computed features used: {cnn_features_path}")

### Load CNN Features (also previously generated)

In [ ]:
# load features + combine the features with labels dataframe
cnn_features = merge_features_with_labels(
    features_path=cnn_features_path,
    labels_df=label,
    export=True,
)

## (B) Heuristic Feature Extraction
Attention! The order of the features using the CNN and this Class is not necessarily the same!

In [ ]:
extractor = ImageHeuristicFeatureExtractor(
    data_folder_path, label.set_index("image_id")
)

feature_label_data = extractor.get_feature_and_label_arrays()
df_heuristic = (
    extractor.return_one_df()
)  # effectively dummy df with the filenames and image ids

x_rgb, y_rgb = feature_label_data["rgb"]
x_hsv, y_hsv = feature_label_data["hsv"]
x_glcm, y_glcm = feature_label_data["glcm"]
# x_gabor, y_gabor = feature_label_data['gabor']

In [ ]:
x_rgb_standardized = standardize_features(x_rgb, use_pca=True, n_components=0.9)
x_hsv_stanardized = standardize_features(x_hsv, use_pca=True, n_components=0.9)

x_heuristic = np.concatenate((x_rgb_standardized, x_hsv_stanardized, x_glcm), axis=1)
y_heuristic = y_hsv
np.shape(x_heuristic)

In [ ]:
names_heuristic_features = []

for i in range(len(x_rgb_standardized[0])):
    names_heuristic_features.append(f"rgb{i}")

for i in range(len(x_hsv_stanardized[0])):
    names_heuristic_features.append(f"hsv{i}")

for i in range(len(x_glcm[0])):
    names_heuristic_features.append(f"glcm{i}")

len(names_heuristic_features)

# Modeling

## Generate the Train and Test Split

In [ ]:
# Define which x and y to use (either run part (A) or (B)

# Heuristic
"""
x = x_heuristic
y = y_heuristic
df_ = df_heuristic
np.shape(x)
"""

# CNN:
x = cnn_features.iloc[:, :1000].to_numpy()
y = cnn_features["cancer"].to_numpy()
df_ = cnn_features

In [ ]:
# only include those files in testing that have not been oversampled
include_in_testing = not_oversampled_images(df_)

TEST_SIZE = 0.20

# Calculate the adjusted test size
test_size_sklearn = calculate_test_size(df_, TEST_SIZE, include_in_testing)

x_train_, x_test, y_train_, y_test = model_selection.train_test_split(
    x[include_in_testing],
    y[include_in_testing],
    test_size=test_size_sklearn,
    random_state=42,
)
x_train = np.concatenate((x_train_, x[np.invert(include_in_testing)]), axis=0)
y_train = np.concatenate((y_train_, y[np.invert(include_in_testing)]), axis=0)

In [ ]:
print(
    f"{len(y_test)} Unique Images used Test Set: {len(y_test) / len(np.unique(df_.image_id)) * 100:.2f}%"
)
print(f"{len(y_train)} Non-Unique Images used Train Set")

## Dimensionality Reduction
### PCA

In [ ]:
# Initialize PCA,
pca = PCA(n_components=0.999)

# Fit and transform the data
pca.fit(np.concatenate((x_train, x_test), axis=0))
x_train_pca = pca.transform(x_train)
x_test_pca = pca.transform(x_test)

# Check the new shape of the data
print(x_train_pca.shape)

In [ ]:
plot_low_dim_components(x_train_pca, y_train, component_1=0, component_2=1)

### Kernel PCA (slow)

In [ ]:
# Initialize PCA,
kpca = KernelPCA(n_components=25, kernel="rbf")  # kernel: rbf, sigmoid

# Fit and transform the data
kpca.fit(np.concatenate((x_train, x_test), axis=0))
x_train_kpca = kpca.transform(x_train)
x_test_kpca = kpca.transform(x_test)

# Check the new shape of the data
print(x_train_kpca.shape)

In [ ]:
plot_low_dim_components(x_train_kpca, y_train, label="kPCA")

### t-SNE (Visualisation *only*)

In [ ]:
# Set the parameters for t-SNE
tsne = TSNE(n_components=2, random_state=0, perplexity=30, n_iter=2000, verbose=1)

# Perform t-SNE on the data
X_tsne = tsne.fit_transform(np.concatenate((x_train, x_test), axis=0))

In [ ]:
plot_low_dim_components(
    X_tsne, np.concatenate((y_train, y_test), axis=0), label="t-SNE"
)

### Use Lower Dimensional Features?

In [ ]:
x_train = x_train  # x_train #x_train_kpca #x_train_pca
x_test = x_test  # x_test #x_test_kpca#x_test_pca

## LogisticRegression

In [ ]:
prediction_model = linear_model.LogisticRegression(
    solver="newton-cg",
    multi_class="auto",
    max_iter=10000,
    class_weight="balanced",
)

prediction_model.fit(x_train, y_train)

y_train_pred = prediction_model.predict(x_train)
y_pred = prediction_model.predict(x_test)

cf = plot_confusion_matrix(y_test, y_pred, return_fig=True)
fig = cf.figure_
plt.gca().set_title("Logistic Regression")
fig.tight_layout()
fig.savefig(figures_folder_path + "/log_regression.pdf")

## SVM

In [ ]:
# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)

# Create an SVM classifier
prediction_model = svm.SVC(
    kernel="poly",
    C=1.0,
    gamma=0.5,
    class_weight="balanced",
)

# Train the classifier
prediction_model.fit(x_train, y_train)

# Make predictions
y_pred = prediction_model.predict(x_test)

cf = plot_confusion_matrix(y_test, y_pred, return_fig=True)
fig = cf.figure_
plt.gca().set_title("SVM")
fig.tight_layout()
fig.savefig(figures_folder_path + "/svm.pdf")

## MLPClassifier

In [ ]:
# solvers = ["lbfgs", "sgd", "adam"]
prediction_model = MLPClassifier(
    hidden_layer_sizes=[400] * 4,
    random_state=1,
    verbose=0,
    solver="adam",
    # learning_rate="adaptive",
)
prediction_model.fit(x_train, y_train)
y_pred = prediction_model.predict(x_test)

cf = plot_confusion_matrix(y_test, y_pred, return_fig=True)
fig = cf.figure_
plt.gca().set_title("MLP")
fig.tight_layout()
fig.savefig(figures_folder_path + "/mlp.pdf")

## AdaBoost Classifer  

In [ ]:
estimator = DecisionTreeClassifier(max_depth=3)
adaboost_model = AdaBoostClassifier(
    estimator=estimator, n_estimators=500, algorithm="SAMME", random_state=0
)
adaboost_model.fit(x_train, y_train)
y_pred = adaboost_model.predict(x_test)

cf = plot_confusion_matrix(y_test, y_pred, return_fig=True)
fig = cf.figure_
plt.gca().set_title("AdaBoost")
fig.tight_layout()
fig.savefig(figures_folder_path + "/adaboost.pdf")

## SHAP Values
We can see how much individual features are influencing the result! Especially useful for the heuristic features

In [ ]:
# Create the SHAP Explainer

# With names (only defined for heuristic features)
"""
explainer = shap.Explainer(
    prediction_model.predict,
    x_train,
    max_evals=4000,
    verbose=1,
    feature_names=names_heuristic_features,
)
"""

# Without names (CNN features)
explainer = shap.Explainer(prediction_model.predict, x_train, max_evals=2500, verbose=1)

shap_values = explainer(x_test)

In [ ]:
fig = plt.figure()
shap.summary_plot(shap_values, x_test, max_display=6)

# Save the current figure
save_path = os.path.join(figures_folder_path, "shap_values.pdf")
fig.savefig(figures_folder_path + "/shap.pdf")

# End to End RestNet(Training and Testing)

## Setup

In [ ]:
model = resnet50(weights=ResNet50_Weights.DEFAULT)
for param in model.parameters():
    param.requires_grad = False

num_classes = 2
model.fc = nn.Linear(model.fc.in_features, num_classes)

### Setup DataLoader

In [ ]:
# Criterion (Loss function)
criterion = nn.CrossEntropyLoss()

# Optimizer (Only train the final layer)
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

# Learning rate scheduler (optional)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
df_img_labels = pd.DataFrame(label[["image_id", "cancer"]])
df_img_labels["result"] = df_img_labels["cancer"].apply(
    lambda x: 0 if x is False else 1
)

# Select only the images that are in the data folder
df_img_labels = df_img_labels[
    df_img_labels.image_id.isin(
        [x.split(".")[-2] for x in os.listdir(data_folder_path)]
    )
]
df_img_labels

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        """
        Args:
            dataframe (DataFrame): DataFrame containing image IDs and labels.
            img_dir (str): Directory where images are stored.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.img_labels = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        self.img_labels.iloc[idx, 0]
        img_path = os.path.join(
            self.img_dir, self.img_labels.iloc[idx, 0] + ".jpg"
        )  # Assuming images are .jpg
        image = Image.open(img_path).convert("RGB")
        label = self.img_labels.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
original_folder_path = data_folder_path
transform = transforms.Compose(
    [
        transforms.Resize(232),
        transforms.CenterCrop(450),  # adapted to use larger region
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)
dataset = CustomImageDataset(
    dataframe=df_img_labels, img_dir=original_folder_path, transform=transform
)

### Splitting Data

In [ ]:
total_size = len(dataset)
test_size = int(0.2 * total_size)
train_size = total_size - test_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

## Training and Testing

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"
print(f"Using device: {device}")

model.to(device)

num_epochs = 10  # Set the number of epochs

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    train_loss = 0.0
    train_corrects = 0

    # Training loop
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Statistics
        train_loss += loss.item() * inputs.size(0)
        train_corrects += torch.sum(preds == labels.data)
        # print(f'Loss: {loss.item()}')

    train_loss /= len(train_loader.dataset)
    train_acc = train_corrects.float() / len(train_loader.dataset)

    # Print training results
    print(
        f"Epoch {epoch+1}/{num_epochs} - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}"
    )

    # Evaluation loop
model.eval()  # Set model to evaluate mode
all_test_preds = []
all_test_labels = []
test_loss = 0.0
test_corrects = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        all_test_preds.extend(preds.cpu().numpy())
        all_test_labels.extend(labels.cpu().numpy())
        # Statistics
        test_loss += loss.item() * inputs.size(0)
        test_corrects += torch.sum(preds == labels.data)

test_loss /= len(test_loader.dataset)
test_acc = test_corrects.float() / len(test_loader.dataset)

In [ ]:
cf = plot_confusion_matrix(all_test_labels, all_test_preds, return_fig=True)
fig = cf.figure_
plt.gca().set_title("ResNet End-to-End ")
fig.tight_layout()
fig.savefig(figures_folder_path + "/resnet_end_to_end.pdf")